In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ProgbarLogger, CSVLogger, Callback
from tensorflow.keras.applications.resnet50 import preprocess_input
import time

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
DATA_DIR = "/content/drive/MyDrive/animal_data"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 15
EPOCHS_STAGE1 = 3
EPOCHS_STAGE2 = 5


In [6]:
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start
        total_time = time.time() - self.start_time
        self.epoch_times.append(epoch_time)

        print(f"\nEpoch {epoch+1} time: {epoch_time:.2f}s | "
              f"Total elapsed: {total_time/60:.2f} min")


In [7]:
time_callback = TimeHistory()
csv_logger = CSVLogger(
    "/content/drive/MyDrive/training_log.csv",
    append=True
)


In [8]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=15,
    horizontal_flip=True,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)


In [9]:
train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.


In [10]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)

outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)



In [12]:
model.compile(
    optimizer=Adam(learning_rate=3e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [13]:
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_STAGE1,
    callbacks=[time_callback, csv_logger],
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4061 - loss: 2.3717
Epoch 1 time: 534.18s | Total elapsed: 8.91 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 534s 11s/step - accuracy: 0.4111 - loss: 2.3500 - val_accuracy: 0.8903 - val_loss: 0.3154
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9277 - loss: 0.2302
Epoch 2 time: 365.33s | Total elapsed: 15.00 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 365s 7s/step - accuracy: 0.9278 - loss: 0.2297 - val_accuracy: 0.9295 - val_loss: 0.2020
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9681 - loss: 0.1055
Epoch 3 time: 331.57s | Total elapsed: 20.52 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 332s 7s/step - accuracy: 0.9680 - loss: 0.1059 - val_accuracy: 0.9347 - val_loss: 0.2305


In [14]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False


In [15]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [16]:
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_STAGE2,
    callbacks=[time_callback, csv_logger],
    verbose=1
)


Epoch 1/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9678 - loss: 0.1123
Epoch 1 time: 416.14s | Total elapsed: 6.94 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 416s 8s/step - accuracy: 0.9679 - loss: 0.1120 - val_accuracy: 0.9556 - val_loss: 0.1541
Epoch 2/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9919 - loss: 0.0422
Epoch 2 time: 402.76s | Total elapsed: 13.65 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 403s 8s/step - accuracy: 0.9919 - loss: 0.0421 - val_accuracy: 0.9634 - val_loss: 0.1357
Epoch 3/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9936 - loss: 0.0299
Epoch 3 time: 403.21s | Total elapsed: 20.37 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 403s 8s/step - accuracy: 0.9936 - loss: 0.0298 - val_accuracy: 0.9634 - val_loss: 0.1347
Epoch 4/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9996 - loss: 0.0180
Epoch 4 time: 399.17s | Total elapsed: 27.02 min
49/49 ━━━━━━━━━━━━━━━━━━━━ 399s 8s/step - accuracy: 0.9995 - loss: 0.0180 - val_accuracy: 0.9634 - val_loss: 0.1386
Epoch 5/5
49/

In [21]:
val_gen_eval = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False   # 🔥 THIS IS THE KEY
)


Found 383 images belonging to 15 classes.


In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [23]:
y_true = val_gen_eval.classes
y_pred_probs = model.predict(val_gen_eval, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy  = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro")
recall    = recall_score(y_true, y_pred, average="macro")
f1        = f1_score(y_true, y_pred, average="macro")

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 64s 5s/step
Accuracy : 0.9634
Precision: 0.9665
Recall   : 0.9635
F1 Score : 0.9634
